In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Define basic data

In [5]:
# define the years to be classified
year_name  = list([f'{span[0]}_{span[1]}' for span in zip(range(1990,2020,3),range(1992,2020,3)) ])

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [6]:
# define the path to point_with_value GEE-Path
path = 'users/Jinzhu_Deakin/North_China_Plain/Sample_with_Landsat_Fourier_Normalized'

# Use the Invarient points to compute the classification accuracy
Invarient_non_built_sample = [ee.FeatureCollection(f'{path}/Invarient_non_built_{year}')\
                                .filterMetadata('Built','equals',0) for year in year_name]
Invarient_built_sample     = [ee.FeatureCollection(f'{path}/Invarient_built_{year}')
                                for year in year_name]
Invarient_sample_merge     = [i[0].merge(i[1]) for i in zip(Invarient_non_built_sample,
                                                            Invarient_built_sample)]

##### Calculate the accuracy of my result at each year

In [7]:
Classificatioin_ACC = {}

for year,sample in zip(year_name,Invarient_sample_merge):
    
    all_bands = list(sample.first().getInfo()['properties'].keys())
    all_bands.remove('Built')
    
    # Instatiate the class with a name.
    classification = Classification(year_name      = year,
                                    Verified_point = sample,
                                    Input_img      = None,
                                    Input_band     = all_bands,
                                    Tree_num       = 100,
                                    Zone_sample    = None,
                                    classProperty  = 'Built')


    # perform the classification on Samples
    classification.Stp_2_Classification_on_Samples()

    # Get the classified samples
    train_sample_classified        = classification.Train_sample_classification
    test_sample_classified         = classification.Test_sample_classification
    
    # Compute the accuracy
    test_acc = Accuracy_assesment(test_sample_classified).Stp_1_Calculate_Accuracy()
    
    # add classfication into the list
    Classificatioin_ACC[year] = test_acc
    
    # print out the process
    print(f'Accuracy computation of {year} complete!')

Accuracy computation of 1990_1992 complete!
Accuracy computation of 1993_1995 complete!
Accuracy computation of 1996_1998 complete!
Accuracy computation of 1999_2001 complete!
Accuracy computation of 2002_2004 complete!
Accuracy computation of 2005_2007 complete!
Accuracy computation of 2008_2010 complete!
Accuracy computation of 2011_2013 complete!
Accuracy computation of 2014_2016 complete!
Accuracy computation of 2017_2019 complete!


In [8]:
My_acc_df = pd.DataFrame(Classificatioin_ACC).T
My_acc_df.index.name = 'Year'

#### Calculate the accuracy of GAIA datasets

In [9]:
GAIA = ee.Image("users/wangjinzhulala/Gloable_Imprevious_area").clip(North_China_Plain)

In [10]:
# link the year with its pixel value
GAIA_year = range(1985,2019)
GAIA_id   = range(34,0,-1)
GAIA_corr = {i[0]:i[1] for i in zip(GAIA_year,GAIA_id)}

Fourier_end_years = list(range(1992,2018,3)) + [2018]

In [11]:
# Here use the Classification class to perform the train/test split
# and get the test samples.

Sample_classificaiton = Classification( # input year_name does not affect train/test split
                                        year_name      = 'any',
    
                                        # Use the first Invariant sample is OK
                                        Verified_point = Invarient_sample_merge[0],
                                        Input_img      = None,
                                        # Just need ['Built'] information
                                        Input_band     = ['Built'],
                                        Tree_num       = 100,
                                        Zone_sample    = None,
                                        classProperty  = 'Built')

test_sample = classification.Verified_pts_test

In [12]:
GAIA_acc = {}

for i,end_year in enumerate(Fourier_end_years):
    
    # get the sample point
    Invariant_sample = Invarient_sample_merge[i]
    
    # get the pixel value of coresponded end_year
    # rename the GAIA to 'classificaiton' so that the name
    # is consistent with the Accuracy_assessment module
    GAIA_pixel_value = GAIA_corr[end_year]
    
    # conver pixel value greater than GAIA_pixel_value to 0,other pixel to 1
    # so we get all target-year pixel with value 1
    GAIA_within_year = GAIA.gt(GAIA_pixel_value).rename('classification')
    
    # extract reclassified GAIA pixel value to test_sample
    test_with_GAIA = GAIA_within_year.sampleRegions(collection = test_sample, 
                                                    properties = ['Built'], 
                                                    scale      = 30)
    # compute the accuracy 
    GAIA_acc[end_year] = Accuracy_assesment(test_with_GAIA).Stp_1_Calculate_Accuracy()
    
    # print out the process
    print(f'Accuracy computation of {end_year} complete!')
    

Accuracy computation of 1992 complete!
Accuracy computation of 1995 complete!
Accuracy computation of 1998 complete!
Accuracy computation of 2001 complete!
Accuracy computation of 2004 complete!
Accuracy computation of 2007 complete!
Accuracy computation of 2010 complete!
Accuracy computation of 2013 complete!
Accuracy computation of 2016 complete!
Accuracy computation of 2018 complete!


In [13]:
GAIA_acc_df = pd.DataFrame(GAIA_acc).T
GAIA_acc_df.index.name = 'End_year'

##### Save the acc_df to local disk

In [14]:
# reset index so can drwo them togeter
# My_acc_df.reset_index(inplace=True)
# GAIA_acc_df.reset_index(inplace=True)

# My_acc_df.to_csv('ACC_Result/My_accuracy.csv',index=False)
# GAIA_acc_df.to_csv('ACC_Result/GAIA_accuracy.csv',index=False)

In [37]:
My_acc_df = pd.read_csv('ACC_Result/My_accuracy.csv')
GAIA_acc_df = pd.read_csv('ACC_Result/GAIA_accuracy.csv')